In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import seed
from collections import Counter
from datetime import datetime
import time
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, cohen_kappa_score, accuracy_score

import sleep_study as ss

# Useful functions

In [2]:
def get_metrics(y, y_hat, verbose=True):
    cf = confusion_matrix(y, y_hat)
    ncf = 100*confusion_matrix(y, y_hat, normalize='true') # percentage
    precision,recall,mf,support = precision_recall_fscore_support(y,y_hat,average='macro')
    kp = cohen_kappa_score(y, y_hat)
    acc = 100*accuracy_score(y, y_hat) # percentage
    
    if verbose:
        print('\nConfusion Matrix')
        print(cf)
        print('\nNormalized Confusion Matrix')
        print(np.round(ncf))
        print('Accuracy: ', acc)
        print('Precision: ',precision)
        print('Recall: ',recall)
        print('Macro-F1 Score: ',mf)
        print('Kohen Kappa: ',kp)
        
    return {'confusion matrix': cf,
            'normalized confusion matrix': ncf,
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'Macro-F1 Score': mf,
            'Kohen Kappa': kp}

In [3]:
def print_all_metrics(res, keys):
    ncf_name = 'normalized confusion matrix'
    
    for i, k in enumerate(keys):
        all_metrics = res[k]
        num_trials = len(all_metrics)
                
        means = pd.DataFrame(data = np.round(np.mean([all_metrics[rr][ncf_name] for rr in range(num_trials)], axis=0), 1), dtype='str')
        stds = pd.DataFrame(data = np.round(np.std([all_metrics[rr][ncf_name] for rr in range(num_trials)], axis=0), 1), dtype='str')
        tbl = (means + " " + u"\u00B1" + " " + stds).to_numpy()
        print(ncf_name)
        print(tbl) # I want normalized CF to print nicely with mean \pm standard deviation.

        for j in ['accuracy', 'precision', 'recall', 'Macro-F1 Score', 'Kohen Kappa']:
            m = str(np.round(np.mean([all_metrics[rr][j] for rr in range(num_trials)]), 3))
            s = str(np.round(np.std([all_metrics[rr][j] for rr in range(num_trials)]), 3))
            print(j, ":", m + " " + u"\u00B1" + " " + s)
        print( )

In [4]:
def get_npz_file_names(path):
    files = []
    with os.scandir(path) as it:
        for entry in it:
            if (entry.name.endswith('.npz') or entry.name.endswith('.NPZ')) and entry.is_file():
                files.append(path+entry.name)
    return files

In [5]:
def load_features(fn, verbose=False):   
    X = np.load(fn, allow_pickle=True)
    features, labels, studies = X["features"], X["labels"], X["studies"]
    if verbose:
        print(fn)
        print(features.shape, labels.shape, len(studies))
        
    return features, labels, studies

In [6]:
def train_test_classifier(idxs, data, clf, verbose=True):
    
    train_idx, test_idx = idxs
    X, y = data
    
    X_train, X_test = X[train_idx,:], X[test_idx,:]
    y_train, y_test = y[train_idx], y[test_idx]
    
    if verbose:
        print('train set class distribution', sorted(Counter(y_train).items()))
        print('test set class distribution', sorted(Counter(y_test).items()))

    t = time.time()
    clf.fit(X_train, y_train)

    if verbose:
        print('training took', np.around((time.time()-t)/60, 2), 'minutes.')

    y_hat = rf.predict(X_test)

    metrics = get_metrics(y_test, y_hat, verbose=False)

    if verbose:
        print(metrics)
        print( )

    return metrics, y_hat

# Feature and label extraction

In [7]:
ss.init()
print('total number of sleep study files available:', len(ss.data.study_list))

age information stored in /home/harlinl/sleep_study_harlin/age_file.csv
total number of sleep study files available: 3984


In [8]:
age_groups = list(zip(range(0, 18), range(1, 19))) + [(18, 100)]

tmp = np.load('study_lists.npz', allow_pickle=True)
study_lists = tmp["study_lists"]
num_segments = tmp["num_segments"]
all_labels = tmp["all_labels"]

In [9]:
# this can be parallelized to speed up the process
for i, study_list in enumerate(study_lists):
    all_features = []
    all_labels = []
    for j, name in enumerate(study_lists[i]):
        features, labels = ss.data.get_demo_wavelet_features_and_labels(name)
        all_features.extend(features)
        all_labels.extend(labels)
    
    features = np.array(all_features)
    labels = np.array(all_labels)
    print( )
    print(features.shape, labels.shape)
    
    fn = 'wavelet_features3/6_dbwt_' + str(age_groups[i][0]) + '_' + str(age_groups[i][1]) + 'yrs_' + \
        datetime.now().isoformat(timespec='minutes') + '.npz'
    
    np.savez(fn, labels=labels, features=features, studies=study_lists[i])
    print('features from', age_groups[i][0], 'to', age_groups[i][1], 'y.o. pts saved in', fn)
    print(' ')


(230824, 7, 12) (230824,)
features from 0 to 1 y.o. pts saved in wavelet_features3/6_dbwt_0_1yrs_2020-12-12T14:25.npz
 

(186088, 7, 12) (186088,)
features from 1 to 2 y.o. pts saved in wavelet_features3/6_dbwt_1_2yrs_2020-12-12T14:35.npz
 

(278268, 7, 12) (278268,)
features from 2 to 3 y.o. pts saved in wavelet_features3/6_dbwt_2_3yrs_2020-12-12T14:47.npz
 

(261939, 7, 12) (261939,)
features from 3 to 4 y.o. pts saved in wavelet_features3/6_dbwt_3_4yrs_2020-12-12T14:57.npz
 

(244173, 7, 12) (244173,)
features from 4 to 5 y.o. pts saved in wavelet_features3/6_dbwt_4_5yrs_2020-12-12T15:07.npz
 

(222787, 7, 12) (222787,)
features from 5 to 6 y.o. pts saved in wavelet_features3/6_dbwt_5_6yrs_2020-12-12T15:16.npz
 

(229099, 7, 12) (229099,)
features from 6 to 7 y.o. pts saved in wavelet_features3/6_dbwt_6_7yrs_2020-12-12T15:25.npz
 

(221072, 7, 12) (221072,)
features from 7 to 8 y.o. pts saved in wavelet_features3/6_dbwt_7_8yrs_2020-12-12T15:35.npz
 

(200103, 7, 12) (200103,)
featu

# Train and test classifier

In [10]:
seed = 0 # for reproducibility
num_trials = 3

rf = RandomForestClassifier(n_estimators=100, random_state=seed, bootstrap=True, n_jobs=-1)
cv = StratifiedKFold(n_splits=num_trials, shuffle=True, random_state=seed)

feature_files = get_npz_file_names('wavelet_features3/')
print(feature_files)

['wavelet_features3/6_dbwt_0_1yrs_2020-12-12T14:25.npz', 'wavelet_features3/6_dbwt_1_2yrs_2020-12-12T14:35.npz', 'wavelet_features3/6_dbwt_2_3yrs_2020-12-12T14:47.npz', 'wavelet_features3/6_dbwt_3_4yrs_2020-12-12T14:57.npz', 'wavelet_features3/6_dbwt_4_5yrs_2020-12-12T15:07.npz', 'wavelet_features3/6_dbwt_5_6yrs_2020-12-12T15:16.npz', 'wavelet_features3/6_dbwt_6_7yrs_2020-12-12T15:25.npz', 'wavelet_features3/6_dbwt_7_8yrs_2020-12-12T15:35.npz', 'wavelet_features3/6_dbwt_8_9yrs_2020-12-12T15:43.npz', 'wavelet_features3/6_dbwt_9_10yrs_2020-12-12T15:51.npz', 'wavelet_features3/6_dbwt_10_11yrs_2020-12-12T15:58.npz', 'wavelet_features3/6_dbwt_11_12yrs_2020-12-12T16:05.npz', 'wavelet_features3/6_dbwt_12_13yrs_2020-12-12T16:12.npz', 'wavelet_features3/6_dbwt_13_14yrs_2020-12-12T16:18.npz', 'wavelet_features3/6_dbwt_14_15yrs_2020-12-12T16:23.npz', 'wavelet_features3/6_dbwt_15_16yrs_2020-12-12T16:30.npz', 'wavelet_features3/6_dbwt_16_17yrs_2020-12-12T16:36.npz', 'wavelet_features3/6_dbwt_17_18y

In [11]:
# concatenate all features and labels 
X, y = [], []
all_studies = []

for fn in feature_files:
    features, labels, studies = load_features(fn)
    print(fn)
    print(features.shape, labels.shape, len(studies))   
    
    X.extend(features.reshape(len(features), -1))
    y.extend(labels)
    all_studies.extend(studies)

X, y = np.array(X), np.array(y)
print(X.shape, y.shape, len(all_studies))


# train and test RF classifier on all data
all_metrics = []

for train_idx, test_idx in cv.split(X, y):

    metrics, y_hat = train_test_classifier((train_idx, test_idx), (X, y), rf)
    all_metrics.append(metrics)

res = {'all ages': all_metrics}

np.savez('wavelet' + '_all_results' + datetime.now().isoformat(timespec='minutes') + '.npz', results=res)

wavelet_features3/6_dbwt_0_1yrs_2020-12-12T14:25.npz
(230824, 7, 12) (230824,) 242
wavelet_features3/6_dbwt_1_2yrs_2020-12-12T14:35.npz
(186088, 7, 12) (186088,) 192
wavelet_features3/6_dbwt_2_3yrs_2020-12-12T14:47.npz
(278268, 7, 12) (278268,) 295
wavelet_features3/6_dbwt_3_4yrs_2020-12-12T14:57.npz
(261939, 7, 12) (261939,) 277
wavelet_features3/6_dbwt_4_5yrs_2020-12-12T15:07.npz
(244173, 7, 12) (244173,) 257
wavelet_features3/6_dbwt_5_6yrs_2020-12-12T15:16.npz
(222787, 7, 12) (222787,) 237
wavelet_features3/6_dbwt_6_7yrs_2020-12-12T15:25.npz
(229099, 7, 12) (229099,) 242
wavelet_features3/6_dbwt_7_8yrs_2020-12-12T15:35.npz
(221072, 7, 12) (221072,) 236
wavelet_features3/6_dbwt_8_9yrs_2020-12-12T15:43.npz
(200103, 7, 12) (200103,) 214
wavelet_features3/6_dbwt_9_10yrs_2020-12-12T15:51.npz
(177709, 7, 12) (177709,) 192
wavelet_features3/6_dbwt_10_11yrs_2020-12-12T15:58.npz
(172860, 7, 12) (172860,) 189
wavelet_features3/6_dbwt_11_12yrs_2020-12-12T16:05.npz
(165875, 7, 12) (165875,) 182

In [12]:
print_all_metrics(res, ['all ages'])

normalized confusion matrix
[['63.1 ± 0.0' '0.0 ± 0.0' '34.0 ± 0.1' '1.5 ± 0.0' '1.4 ± 0.0']
 ['23.9 ± 0.2' '0.9 ± 0.0' '68.1 ± 0.3' '2.1 ± 0.0' '5.0 ± 0.1']
 ['4.4 ± 0.0' '0.0 ± 0.0' '88.6 ± 0.0' '5.8 ± 0.0' '1.1 ± 0.0']
 ['1.7 ± 0.0' '0.0 ± 0.0' '27.2 ± 0.1' '70.7 ± 0.1' '0.4 ± 0.0']
 ['6.7 ± 0.0' '0.0 ± 0.0' '76.6 ± 0.0' '1.5 ± 0.0' '15.1 ± 0.0']]
accuracy : 64.38 ± 0.025
precision : 0.715 ± 0.002
recall : 0.477 ± 0.0
Macro-F1 Score : 0.48 ± 0.0
Kohen Kappa : 0.482 ± 0.0



In [13]:
res = {}
# train and test RF classifier on each age group

for fn in feature_files:
    X, y, studies = load_features(fn)
    X = X.reshape(len(y), -1)
    print(fn)
    print(X.shape, y.shape, len(studies))
    
    all_metrics = []
    
    for train_idx, test_idx in cv.split(X, y):
        
        metrics, y_hat = train_test_classifier((train_idx, test_idx), (X, y), rf, verbose=False)
        all_metrics.append(metrics)

    res[fn] = all_metrics
        
np.savez('wavelet' + '_age_groups_results' + datetime.now().isoformat(timespec='minutes') + '.npz', results=res)    

wavelet_features3/6_dbwt_0_1yrs_2020-12-12T14:25.npz
(230824, 84) (230824,) 242
wavelet_features3/6_dbwt_1_2yrs_2020-12-12T14:35.npz
(186088, 84) (186088,) 192
wavelet_features3/6_dbwt_2_3yrs_2020-12-12T14:47.npz
(278268, 84) (278268,) 295
wavelet_features3/6_dbwt_3_4yrs_2020-12-12T14:57.npz
(261939, 84) (261939,) 277
wavelet_features3/6_dbwt_4_5yrs_2020-12-12T15:07.npz
(244173, 84) (244173,) 257
wavelet_features3/6_dbwt_5_6yrs_2020-12-12T15:16.npz
(222787, 84) (222787,) 237
wavelet_features3/6_dbwt_6_7yrs_2020-12-12T15:25.npz
(229099, 84) (229099,) 242
wavelet_features3/6_dbwt_7_8yrs_2020-12-12T15:35.npz
(221072, 84) (221072,) 236
wavelet_features3/6_dbwt_8_9yrs_2020-12-12T15:43.npz
(200103, 84) (200103,) 214
wavelet_features3/6_dbwt_9_10yrs_2020-12-12T15:51.npz
(177709, 84) (177709,) 192
wavelet_features3/6_dbwt_10_11yrs_2020-12-12T15:58.npz
(172860, 84) (172860,) 189
wavelet_features3/6_dbwt_11_12yrs_2020-12-12T16:05.npz
(165875, 84) (165875,) 182
wavelet_features3/6_dbwt_12_13yrs_2